# NOTE:  DO NOT RUN THIS NOTEBOOK UNTIL THE PREVIOUS NOTEBOOK IS COMPLETE (~30 MINUTES)

# PLEASE BE PATIENT.

# Predict Customer Reviews with the Autopilot Model

In [1]:
import boto3
import sagemaker
import pandas as pd
import json

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
%store -r autopilot_endpoint_name

In [3]:
try:
    autopilot_endpoint_name
    print("[OK]")
except NameError:
    print("***************************************************************************")
    print("[ERROR] PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH *******************")
    print("[ERROR] OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************")
    print("***************************************************************************")

[OK]


In [4]:
print(autopilot_endpoint_name)

automl-dm-ep-18-20-33-41


# Wait for the Model to Deploy
This may take 5-10 mins.  Please be patient.

In [5]:
autopilot_endpoint_name

'automl-dm-ep-18-20-33-41'

In [6]:
sm.get_waiter("endpoint_in_service").wait(EndpointName=autopilot_endpoint_name)

In [7]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_name)
status = resp["EndpointStatus"]

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Arn: arn:aws:sagemaker:us-east-1:211125778552:endpoint/automl-dm-ep-18-20-33-41
Status: InService


# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [8]:
sm_runtime = boto3.client("sagemaker-runtime")

/home/sagemaker-user/.conda/envs/data_science_on_aws/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [9]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(
    EndpointName=autopilot_endpoint_name, ContentType="text/csv", Accept="text/csv", Body=csv_line_predict_positive
)

response_body = response["Body"].read().decode("utf-8").strip()

r = response_body.split(",")
print("Predicated Star Rating Class: {} \nProbability: {} ".format(r[0], r[1]))

Predicated Star Rating Class: 5 
Probability: 0.37442246079444885 


In [10]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(
    EndpointName=autopilot_endpoint_name, ContentType="text/csv", Accept="text/csv", Body=csv_line_predict_meh
)

response_body = response["Body"].read().decode("utf-8").strip()

r = response_body.split(",")
print("Predicated Star Rating Class: {} \nProbability: {} ".format(r[0], r[1]))

Predicated Star Rating Class: 3 
Probability: 0.27331432700157166 


In [11]:
csv_line_predict_negative = """It's pretty good."""

response = sm_runtime.invoke_endpoint(
    EndpointName=autopilot_endpoint_name, ContentType="text/csv", Accept="text/csv", Body=csv_line_predict_negative
)

response_body = response["Body"].read().decode("utf-8").strip()

r = response_body.split(",")
print("Predicated Star Rating Class: {} \nProbability: {} ".format(r[0], r[1]))

Predicated Star Rating Class: 3 
Probability: 0.24447201192378998 


# Summary
We used Autopilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

Autopilot uses a transparent approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the Autopilot service.

# Release Resources

In [12]:
%store -r autopilot_model_name
print(autopilot_model_name)

automl-dm-model-18-20-33-40


In [13]:
try:
    sm.delete_endpoint(EndpointName=autopilot_endpoint_name)
    print(f'Deleted autopilot endpoint at {autopilot_endpoint_name}')

    sm.delete_model(ModelName=autopilot_model_name)
    print(f'Deleted autopilot model at {autopilot_model_name}')
    
    %store -d autopilot_endpoint_name
    %store -d autopilot_model_name
    %store -d auto_ml_job_name
except Exception as error:
    print(error)

Deleted autopilot endpoint at automl-dm-ep-18-20-33-41
Deleted autopilot model at automl-dm-model-18-20-33-40


In [14]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [15]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>